# Solving FetchReach with A2C


In [ ]:
#
# Installing Stable Baselines3
#
!pip install stable-baselines3 gymnasium-robotics mujoco -q
!apt-get install -y xvfb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

import numpy as np
import gymnasium as gym
import gymnasium_robotics

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import CheckpointCallback

from gym.wrappers import RecordVideo
from IPython.display import Video
import glob

In [ ]:
print(gym.envs.registry.keys())
print(gym.envs.registry["FetchReach-v3"])

dict_keys(['CartPole-v0', 'CartPole-v1', 'MountainCar-v0', 'MountainCarContinuous-v0', 'Pendulum-v1', 'Acrobot-v1', 'phys2d/CartPole-v0', 'phys2d/CartPole-v1', 'phys2d/Pendulum-v0', 'LunarLander-v3', 'LunarLanderContinuous-v3', 'BipedalWalker-v3', 'BipedalWalkerHardcore-v3', 'CarRacing-v3', 'Blackjack-v1', 'FrozenLake-v1', 'FrozenLake8x8-v1', 'CliffWalking-v0', 'Taxi-v3', 'tabular/Blackjack-v0', 'tabular/CliffWalking-v0', 'Reacher-v2', 'Reacher-v4', 'Reacher-v5', 'Pusher-v2', 'Pusher-v4', 'Pusher-v5', 'InvertedPendulum-v2', 'InvertedPendulum-v4', 'InvertedPendulum-v5', 'InvertedDoublePendulum-v2', 'InvertedDoublePendulum-v4', 'InvertedDoublePendulum-v5', 'HalfCheetah-v2', 'HalfCheetah-v3', 'HalfCheetah-v4', 'HalfCheetah-v5', 'Hopper-v2', 'Hopper-v3', 'Hopper-v4', 'Hopper-v5', 'Swimmer-v2', 'Swimmer-v3', 'Swimmer-v4', 'Swimmer-v5', 'Walker2d-v2', 'Walker2d-v3', 'Walker2d-v4', 'Walker2d-v5', 'Ant-v2', 'Ant-v3', 'Ant-v4', 'Ant-v5', 'Humanoid-v2', 'Humanoid-v3', 'Humanoid-v4', 'Humanoid-v5

In [ ]:
# Enable EGL rendering
os.environ['MUJOCO_GL'] = 'egl'

# Create and wrap environment
env = gym.make("FetchReach-v3", reward_type="dense", render_mode="rgb_array")

In [ ]:
# Set up callback for saving checkpoints
checkpoint_callback = CheckpointCallback(
    save_freq=50000,
    save_path="./fetchreach_a2c_checkpoints/",
    name_prefix="fetchreach_a2c_model"
)

In [ ]:
#
# Train the model
#
TOTAL_TIMESTEPS = 1_000_000
model = A2C("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=checkpoint_callback)

# Save the model
model.save("/content/drive/MyDrive/fetchreach_a2c_model_1M") # it's a zip file
env.close()

Streaming output truncated to the last 5000 lines.
|    std                | 0.914    |
|    value_loss         | 0.000169 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 50       |
|    ep_rew_mean        | -12.4    |
|    success_rate       | 0        |
| time/                 |          |
|    fps                | 208      |
|    iterations         | 173800   |
|    time_elapsed       | 4176     |
|    total_timesteps    | 869000   |
| train/                |          |
|    entropy_loss       | -5.14    |
|    explained_variance | 0.987    |
|    learning_rate      | 0.0007   |
|    n_updates          | 173799   |
|    policy_loss        | -0.127   |
|    std                | 0.911    |
|    value_loss         | 0.000854 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 50       |
|    ep_rew_mean        

In [ ]:
#
# Evaluate the trained model
#
env = gym.make("FetchReach-v3", reward_type="dense", render_mode=None)
# Load back model (if necessary)
model = A2C.load("/content/drive/MyDrive/fetchreach_a2c_model_1M")

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(f"\n[Evaluate_Policy] Mean reward: {mean_reward:.2f} ± {std_reward:.2f}")

env.close()


[Evaluate_Policy] Mean reward: -12.47 ± 2.70


In [ ]:
# Manual evaluation over 100 episodes with success rate
env = gym.make("FetchReach-v3", reward_type="dense", render_mode=None)
manual_rewards = []

for ep in range(100):
    obs, _ = env.reset()
    done = False
    episode_reward = 0
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        episode_reward += reward
    manual_rewards.append(episode_reward)

manual_mean = np.mean(manual_rewards)
manual_std = np.std(manual_rewards)

print(f"[Manual Evaluation] Mean reward: {manual_mean:.2f} ± {manual_std:.2f}")

# Comparison
print("\n--- Comparison ---")
print(f"evaluate_policy(): Mean = {mean_reward:.2f}, Std = {std_reward:.2f}")
print(f"Manual rollout:   Mean = {manual_mean:.2f}, Std = {manual_std:.2f}")

env.close()

[Manual Evaluation] Mean reward: -12.59 ± 2.86

--- Comparison ---
evaluate_policy(): Mean = -12.47, Std = 2.70
Manual rollout:   Mean = -12.59, Std = 2.86


**Video Generation**

In [ ]:
# Simulate one trajectory with video recording
env = gym.make("FetchReach-v3", reward_type="dense", render_mode="rgb_array")
env = RecordVideo(env, video_folder="video", episode_trigger=lambda x: True)

obs, _ = env.reset()
achieved, desired = [], []
total_reward = 0
min_distance = float("inf")

for _ in range(100):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    total_reward += reward

    # Track goals
    ag = obs["achieved_goal"]
    dg = obs["desired_goal"]
    achieved.append(ag)
    desired.append(dg)

    # Compute distance
    dist = np.linalg.norm(ag - dg)
    min_distance = min(min_distance, dist)

    if done:
        break

env.close()

achieved = np.array(achieved)
desired = np.array(desired)
total_dist = np.sum(np.linalg.norm(achieved - desired, axis=1))

print(f"Trajectory total reward = {total_reward:.4f}")
print(f"Total Euclidean distance = {total_dist:.4f}")
print(f"Match: {np.isclose(total_reward, -total_dist)}")
print(f"Minimum distance to goal during trajectory = {min_distance:.4f}")


/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Trajectory total reward = -15.7360
Total Euclidean distance = 15.7360
Match: True
Minimum distance to goal during trajectory = 0.2821


In [ ]:
# Show the video
mp4list = glob.glob("video/*.mp4")
mp4list.sort()
Video(mp4list[-1], embed=True)